In [1]:
from metadrive.envs.metadrive_env import MetaDriveEnv
from utils.scenario_runner import ScenarioRunner
from pathlib import Path
import numpy as np

In [2]:
def get_dt(fps):
    # Assume default value
    DECISION_REPEAT = 5
    interval = 1 / fps
    dt = interval / DECISION_REPEAT
    return dt


get_dt(30)

0.006666666666666666

In [3]:
SAVE_DIR = Path("/media/olek/2TB_HDD/metadrive-data/sandbox")

In [4]:
seed = 0
FPS = 60

dt = get_dt(FPS)
sr = ScenarioRunner(SAVE_DIR, seed, dt=dt)

env = MetaDriveEnv(config=sr.get_config())
_, reset_info = env.reset()

[INFO] Environment: MetaDriveEnv
[INFO] MetaDrive version: 0.4.2.3
[INFO] Sensors: [lidar: Lidar(), side_detector: SideDetector(), lane_line_detector: LaneLineDetector()]
[INFO] Render Mode: none
[INFO] Horizon (Max steps per agent): None
[INFO] Assets version: 0.4.2.3
[INFO] Known Pipes: glxGraphicsPipe
[INFO] Start Scenario Index: 0, Num Scenarios : 1


In [33]:
from metadrive.examples.ppo_expert.numpy_expert import expert


action = expert(env.agent, deterministic=True)
obs, reward, terminated, truncated, info = env.step(action)

In [35]:
from abc import ABCMeta
import json


class MetaDriveJSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.float32):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        if isinstance(obj, type):
            return obj.__name__
        return super().default(obj)


def get_bv_state(env) -> dict:
    vehicles = env.agent_manager.get_objects()

    bv_state = {k: v.get_state() for k, v in vehicles.items() if k != env.agent.id}

    return bv_state


step_info = {}

step_info["bv_state"] = get_bv_state(env)
# Serialize bv_state
step_info["bv_state"]

json.dumps(info, cls=MetaDriveJSONEncoder)

'{"navigation_command": "forward", "navigation_forward": true, "navigation_left": false, "navigation_right": false, "overtake_vehicle_num": 0, "velocity": 0.6382159993715163, "steering": -0.11324820667505264, "acceleration": 1.0, "step_energy": 0.0003333486314078067, "episode_energy": 0.0023356600369475395, "policy": "EnvInputPolicy", "action": [-0.11324820667505264, 1.0], "raw_action": [-0.11324820667505264, 1.0], "crash_vehicle": false, "crash_object": false, "crash_building": false, "crash_human": false, "crash_sidewalk": false, "out_of_road": false, "arrive_dest": false, "max_step": false, "env_seed": 0, "crash": false, "step_reward": 0.012885552319437448, "route_completion": 0.007535720517044892, "cost": 0, "episode_reward": 0.09202958937293529, "episode_length": 14}'